In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import os
import seaborn as sns
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
#Read Dataset
dataset = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
dataset_test = pd.read_csv('../input/spaceship-titanic/test.csv')

In [ ]:
dataset.info()

In [ ]:
dataset_test.info()

In [ ]:
X = dataset.iloc[:,:-1]
y = dataset.iloc[:,-1]

In [ ]:
#Dropping unwanted columns
X_test = dataset_test
X = X.drop(columns = ["PassengerId","Name"])
X_test = X_test.drop(columns = ["PassengerId","Name"])
X.columns

In [ ]:
X_test.columns

In [ ]:
#Checking null values in data in % of total values
print(X.isnull().sum()/len(X)*100)
print(X_test.isnull().sum()/len(X)*100)

In [ ]:
#Filling null values with mean for int64 and float64 and back and fwd values in object data
for i in X.columns:
    if X[i].dtype not in ['int64','float64']:
        X[i] = X[i].ffill().bfill()
    else:
        X[i] = X[i].fillna(X[i].mean())
print(X.isnull().sum()/len(X)*100)
for i in X_test.columns:
    if X_test[i].dtype not in ['int64','float64']:
        X_test[i] = X_test[i].ffill().bfill()
    else:
        X_test[i] = X_test[i].fillna(X_test[i].mean())
print(X_test.isnull().sum()/len(X_test)*100)

In [ ]:
#Splitting Cabin column to 3 different data columns
X[["Deck","Num","Side"]] = X["Cabin"].str.split('/', expand = True)
X_test[["Deck","Num","Side"]] = X_test["Cabin"].str.split('/', expand = True)
print(X)
print(X_test)

In [ ]:
X['Side']

In [ ]:
del X['Cabin']
del X_test['Cabin']

In [ ]:
#Typecasting 
X['CryoSleep'] = X['CryoSleep'].astype('bool')
X['VIP'] = X['VIP'].astype('bool')
X_test['CryoSleep'] = X_test['CryoSleep'].astype('bool')
X_test['VIP'] = X_test['VIP'].astype('bool')

In [ ]:
del X['Num']
del X_test['Num']
del X['Deck']
del X_test['Deck']

In [ ]:
print(X)
print(X_test)

In [ ]:
#Identifying colummns to be Label Encoded
categorical_cols = X.select_dtypes(include=['object','bool']).columns.tolist()
categorical_cols_test = X_test.select_dtypes(include=['object','bool']).columns.tolist()

In [ ]:
#Concatinationg treain and test dataset
X['train'] = 1
X_test['train'] = 0
df = pd.concat([X,X_test])
df

In [ ]:
df[categorical_cols].info

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [ ]:
#Applying Label Encoding
df[categorical_cols]= df[categorical_cols].apply(lambda col: le.fit_transform(col))
#Normalizing data in dataframe
df.iloc[:,5:10] = df.iloc[:,5:10].apply(lambda x: (x-x.mean())/ x.std(), axis=0)

In [ ]:
X_test[categorical_cols]

In [ ]:
X_test

In [ ]:
#from sklearn.preprocessing import OneHotEncoder
#ohe = OneHotEncoder() 

In [ ]:
#X_ohe = pd.DataFrame(ohe.fit_transform(X).toarray())
#X_ohe_test = pd.DataFrame(ohe.transform(X_test).toarray())
#df_ohe = pd.DataFrame(ohe.fit_transform(df).toarray())

In [ ]:
#df = df.join(df_ohe)

In [ ]:
#Splitting into train and test dataset
X = df[df['train']==1]
X_test = df[df['train']==0]
print(X.columns)
X = X.drop(columns = ['train'])
X_test = X_test.drop(columns = ['train'])
#X = X.join(X_ohe)
#X_test = X_test.join(X_ohe_test)

In [ ]:
X.info()

In [ ]:
#Loading Model
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = 1000,random_state = 42)

In [ ]:
#Normalizing data in dataframe and checking if there is any null value
#X.iloc[:,5:10] = X.iloc[:,5:10].apply(lambda x: (x-x.mean())/ x.std(), axis=0)

In [ ]:
#X_test.iloc[:,5:10] = X_test.iloc[:,5:10].apply(lambda x: (x-x.mean())/ x.std(), axis=0)

In [ ]:
#Model fitting
rf.fit(X, y)

In [ ]:
score = rf.score(X, y)
print(score)

In [ ]:
predictions = rf.predict(X_test)

In [ ]:
pred = []
for i in predictions:
    if i<0.5:
        pred.append('False')
    else:
        pred.append('True')

In [ ]:
df = pd.DataFrame(pred)
df.to_csv('result.csv')

In [ ]:
df.to_csv('result.csv')